In [ ]:
# 统计60sdf=180fs下的主峰角动量

import sdf_helper as sh
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import os

# # 设定数据文件的路径和文件名模式  
base_path = '/media/lan/4248e56e-6d9a-4026-afad-e8b1d59ceab0/epoch/epoch3d/Cpf-Work/cone/cone1-a=145-0.5微米'
file_prefix = 'weight'  
file_suffix = '.sdf'

file_index =  60

file_name = f"{file_prefix}{file_index:04d}{file_suffix}"  
file_path = os.path.join(base_path, file_name)

# # 读取数据
# den_crit =  0.1
weight_Data = sh.getdata(file_path)
# sh.list_variables(weight_Data)

weight_ph = weight_Data.Particles_Weight_subset_testp_Photon.data
weight_ele = weight_Data.Particles_Weight_subset_teste_Electron.data

In [6]:
# 90666  有问题.这里包含了反射靶
import sdf_helper as sh
import numpy as np
import os

base_path = '/media/lan/4248e56e-6d9a-4026-afad-e8b1d59ceab0/epoch/epoch3d/Cpf-Work/cone/cone1-a=145-3微米'
file_index = 60
file_path = os.path.join(base_path, f'idall{file_index:04d}.sdf')

data = sh.getdata(file_path)
sh.list_variables(data)

def get_data(name):
    arr = getattr(data, name).data
    return arr / 1e-6 if 'Grid_Particles' in name else arr

px_ph, py_ph, pz_ph = (get_data(f'Particles_P{d}_subset_testp_Photon') for d in ['x','y','z'])
x_ph = data.Grid_Particles_subset_testp_Photon.data[0]/1e-6
y_ph = data.Grid_Particles_subset_testp_Photon.data[1]/1e-6
z_ph = data.Grid_Particles_subset_testp_Photon.data[2]/1e-6

px_ele, py_ele, pz_ele = (get_data(f'Particles_P{d}_subset_teste_Electron') for d in ['x','y','z'])
x_ele = data.Grid_Particles_subset_teste_Electron.data[0]/1e-6
y_ele = data.Grid_Particles_subset_teste_Electron.data[1]/1e-6
z_ele = data.Grid_Particles_subset_teste_Electron.data[2]/1e-6


weight_ph, weight_ele = 90666, 90666

x_range = (46.7, 49.1)
y_range = (-10,10)
z_range = (-10,10)

def filter_range(x, y, z, xr, yr, zr):
    mask = np.ones_like(x, dtype=bool)
    if xr: mask &= (x >= xr[0]) & (x <= xr[1])
    if yr: mask &= (y >= yr[0]) & (y <= yr[1])
    if zr: mask &= (z >= zr[0]) & (z <= zr[1])
    return mask

mask_ph = filter_range(x_ph, y_ph, z_ph, x_range, y_range, z_range)
mask_ele = filter_range(x_ele, y_ele, z_ele, x_range, y_range, z_range)

Lx_ph = (y_ph[mask_ph] * pz_ph[mask_ph] - z_ph[mask_ph] * py_ph[mask_ph]) * weight_ph
Lx_ele = (y_ele[mask_ele] * pz_ele[mask_ele] - z_ele[mask_ele] * py_ele[mask_ele]) * weight_ele

# 打印筛选后的粒子坐标范围和对应角动量统计信息
def print_stats(particle_type, x, y, z, Lx, mask):
    print(f"{particle_type} 粒子筛选后范围及角动量统计：")
    print(f"x 范围: [{x[mask].min():.3f}, {x[mask].max():.3f}] μm")
    print(f"y 范围: [{y[mask].min():.3f}, {y[mask].max():.3f}] μm")
    print(f"z 范围: [{z[mask].min():.3f}, {z[mask].max():.3f}] μm")
    print(f"Lx 统计:")
    print(f"  最小值: {Lx.min():.3e}")
    print(f"  最大值: {Lx.max():.3e}")
    print(f"  平均值: {Lx.mean():.3e}")
    print(f"  标准差: {Lx.std():.3e}")
    print(f"  总和:   {Lx.sum():.3e}")
    print("-" * 40)


print_stats('光子', x_ph, y_ph, z_ph, Lx_ph, mask_ph)
print_stats('电子', x_ele, y_ele, z_ele, Lx_ele, mask_ele)

Reading file /media/lan/4248e56e-6d9a-4026-afad-e8b1d59ceab0/epoch/epoch3d/Cpf-Work/cone/cone1-a=145-3微米/idall0060.sdf
t() = time
px_subset_teste_electron(77701093,) = Particles/Px/subset_teste/Electron
py_subset_teste_electron(77701093,) = Particles/Py/subset_teste/Electron
pz_subset_teste_electron(77701093,) = Particles/Pz/subset_teste/Electron
ek_subset_teste_electron(77701093,) = Particles/Ek/subset_teste/Electron
id_subset_teste_electron(77701093,) = Particles/ID/subset_teste/Electron
x_grid_subset_teste_electron(77701093, 77701093, 77701093) = Grid_Particles_subset_teste_Electron x
y_grid_subset_teste_electron(77701093, 77701093, 77701093) = Grid_Particles_subset_teste_Electron y
z_grid_subset_teste_electron(77701093, 77701093, 77701093) = Grid_Particles_subset_teste_Electron z
px_subset_testp_photon(1264859,) = Particles/Px/subset_testp/Photon
py_subset_testp_photon(1264859,) = Particles/Py/subset_testp/Photon
pz_subset_testp_photon(1264859,) = Particles/Pz/subset_testp/Photon
e

In [ ]:
import sdf_helper as sh
import numpy as np
import pandas as pd
import os

# ---------- 基本设置 ----------
base_path = '/media/lan/4248e56e-6d9a-4026-afad-e8b1d59ceab0/epoch/epoch3d/Cpf-Work/cone/cone1-a=145-0.5微米'
file_index = 60  # 你想处理的文件编号
w0 = 7e-6  

# ---------- 构建文件路径 ----------
file_name = f"idall{file_index:04d}.sdf"
file_name_w = f"weight{file_index:04d}.sdf"
file_path = os.path.join(base_path, file_name)
file_path_w = os.path.join(base_path, file_name_w)

if not os.path.exists(file_path) or not os.path.exists(file_path_w):
    raise FileNotFoundError(f"{file_name} 或 {file_name_w} 不存在")

# ---------- 读取数据 ----------
Data_idall = sh.getdata(file_path)
Data_w = sh.getdata(file_path_w)

# ---------- 电子数据 ----------
ex = Data_idall.Grid_Particles_subset_teste_Electron.data[0]
ey = Data_idall.Grid_Particles_subset_teste_Electron.data[1]
ez = Data_idall.Grid_Particles_subset_teste_Electron.data[2]
epx = Data_idall.Particles_Px_subset_teste_Electron.data
epy = Data_idall.Particles_Py_subset_teste_Electron.data
epz = Data_idall.Particles_Pz_subset_teste_Electron.data
ew = Data_w.Particles_Weight_subset_teste_Electron.data
ek = Data_idall.Particles_Ek_subset_teste_Electron.data

x0 = 40e-6 + 2e-6 * np.exp(-(ey**2 + ez**2) / w0**2)
emask = (ex >= 20e-6) & (ex <= x0)
Lx_e = np.sum((ey[emask] * epz[emask] - ez[emask] * epy[emask]) * ew[emask])
Ly_e = np.sum((ez[emask] * epx[emask] - ex[emask] * epz[emask]) * ew[emask])
Lz_e = np.sum((ex[emask] * epy[emask] - ey[emask] * epx[emask]) * ew[emask])
Ek_electron = np.sum(ek[emask] * ew[emask])
count_e = np.sum(ew[emask])

# ---------- 光子数据 ----------
try:
    px = Data_idall.Grid_Particles_subset_testp_Photon.data[0]
    py = Data_idall.Grid_Particles_subset_testp_Photon.data[1]
    pz = Data_idall.Grid_Particles_subset_testp_Photon.data[2]
    ppx = Data_idall.Particles_Px_subset_testp_Photon.data
    ppy = Data_idall.Particles_Py_subset_testp_Photon.data
    ppz = Data_idall.Particles_Pz_subset_testp_Photon.data
    pw = Data_w.Particles_Weight_subset_testp_Photon.data
    pek = Data_idall.Particles_Ek_subset_testp_Photon.data

    px0 = 40 + 2 * np.exp(-(py**2 + pz**2) / w0**2)
    pmask = (px >= 20e-6) & (px <= px0 * 1e-6)
    Lx_p = np.sum((py[pmask] * ppz[pmask] - pz[pmask] * ppy[pmask]) * pw[pmask])
    Ly_p = np.sum((pz[pmask] * ppx[pmask] - px[pmask] * ppz[pmask]) * pw[pmask])
    Lz_p = np.sum((px[pmask] * ppy[pmask] - py[pmask] * ppx[pmask]) * pw[pmask])
    Ek_photon = np.sum(pek[pmask] * pw[pmask])
    count_p = np.sum(pw[pmask])
except AttributeError:
    Lx_p = Ly_p = Lz_p = Ek_photon = count_p = 0

# ---------- 能量 ----------
Etot_particle = Data_idall.Total_Particle_Energy_in_Simulation__J_.data / 1.602e-13
Etot_laser = Data_idall.Absorption_Total_Laser_Energy_Injected__J_.data / 1.602e-13

# ---------- 导出 CSV ----------
result = pd.DataFrame([{
    'File_Index': file_index,
    'Lx_Electron': Lx_e, 'Ly_Electron': Ly_e, 'Lz_Electron': Lz_e,
    'Lx_Photon': Lx_p, 'Ly_Photon': Ly_p, 'Lz_Photon': Lz_p,
    'Total_Particle_Energy': Etot_particle,
    'Total_Laser_Energy_Injected': Etot_laser,
    'Electron_Kinetic_Energy': Ek_electron,
    'Photon_Kinetic_Energy': Ek_photon,
    'Electron_Count': count_e,
    'Photon_Count': count_p
}])

save_path = f"角动量结果_idall{file_index:04d}.csv"
result.to_csv(save_path, index=False)
print(f"单帧数据处理完成，结果已保存为 {save_path}")


In [ ]:
import sdf_helper as sh
import numpy as np
import pandas as pd
import os

# ---------- 基本设置 ----------
base_path = '/media/lan/4248e56e-6d9a-4026-afad-e8b1d59ceab0/epoch/epoch3d/Cpf-Work/cone/cone1-a=145-0.5微米'
file_index = 60  # 你想处理的文件编号
w0 = 7e-6  

# ---------- 构建文件路径 ----------
file_name = f"idall{file_index:04d}.sdf"
file_name_w = f"weight{file_index:04d}.sdf"
file_path = os.path.join(base_path, file_name)
file_path_w = os.path.join(base_path, file_name_w)

if not os.path.exists(file_path) or not os.path.exists(file_path_w):
    raise FileNotFoundError(f"{file_name} 或 {file_name_w} 不存在")

# ---------- 读取数据 ----------
Data_idall = sh.getdata(file_path)
Data_w = sh.getdata(file_path_w)

# ---------- 电子数据 ----------
ex = Data_idall.Grid_Particles_subset_teste_Electron.data[0]
ey = Data_idall.Grid_Particles_subset_teste_Electron.data[1]
ez = Data_idall.Grid_Particles_subset_teste_Electron.data[2]
epx = Data_idall.Particles_Px_subset_teste_Electron.data
epy = Data_idall.Particles_Py_subset_teste_Electron.data
epz = Data_idall.Particles_Pz_subset_teste_Electron.data
ew = Data_w.Particles_Weight_subset_teste_Electron.data
ek = Data_idall.Particles_Ek_subset_teste_Electron.data

# x0 = 40e-6 + 2e-6 * np.exp(-(ey**2 + ez**2) / w0**2)
# emask = (ex >= 20e-6) & (ex <= x0)
emask = (ex >= 20e-6) & (ex <= 50e-6)
Lx_e = np.sum((ey[emask] * epz[emask] - ez[emask] * epy[emask]) * ew[emask])
Ly_e = np.sum((ez[emask] * epx[emask] - ex[emask] * epz[emask]) * ew[emask])
Lz_e = np.sum((ex[emask] * epy[emask] - ey[emask] * epx[emask]) * ew[emask])
Ek_electron = np.sum(ek[emask] * ew[emask])
count_e = np.sum(ew[emask])

# ---------- 光子数据 ----------
try:
    px = Data_idall.Grid_Particles_subset_testp_Photon.data[0]
    py = Data_idall.Grid_Particles_subset_testp_Photon.data[1]
    pz = Data_idall.Grid_Particles_subset_testp_Photon.data[2]
    ppx = Data_idall.Particles_Px_subset_testp_Photon.data
    ppy = Data_idall.Particles_Py_subset_testp_Photon.data
    ppz = Data_idall.Particles_Pz_subset_testp_Photon.data
    pw = Data_w.Particles_Weight_subset_testp_Photon.data
    pek = Data_idall.Particles_Ek_subset_testp_Photon.data

    # px0 = 40 + 2 * np.exp(-(py**2 + pz**2) / w0**2)
    # pmask = (px >= 20e-6) & (px <= px0 * 1e-6)
    pmask = (px >= 20e-6) & (px <= px0 * 50e-6)
    Lx_p = np.sum((py[pmask] * ppz[pmask] - pz[pmask] * ppy[pmask]) * pw[pmask])
    Ly_p = np.sum((pz[pmask] * ppx[pmask] - px[pmask] * ppz[pmask]) * pw[pmask])
    Lz_p = np.sum((px[pmask] * ppy[pmask] - py[pmask] * ppx[pmask]) * pw[pmask])
    Ek_photon = np.sum(pek[pmask] * pw[pmask])
    count_p = np.sum(pw[pmask])
except AttributeError:
    Lx_p = Ly_p = Lz_p = Ek_photon = count_p = 0

# ---------- 能量 ----------
Etot_particle = Data_idall.Total_Particle_Energy_in_Simulation__J_.data / 1.602e-13
Etot_laser = Data_idall.Absorption_Total_Laser_Energy_Injected__J_.data / 1.602e-13

# ---------- 导出 CSV ----------
result = pd.DataFrame([{
    'File_Index': file_index,
    'Lx_Electron': Lx_e, 'Ly_Electron': Ly_e, 'Lz_Electron': Lz_e,
    'Lx_Photon': Lx_p, 'Ly_Photon': Ly_p, 'Lz_Photon': Lz_p,
    'Total_Particle_Energy': Etot_particle,
    'Total_Laser_Energy_Injected': Etot_laser,
    'Electron_Kinetic_Energy': Ek_electron,
    'Photon_Kinetic_Energy': Ek_photon,
    'Electron_Count': count_e,
    'Photon_Count': count_p
}])

save_path = f"角动量结果_idall{file_index:04d}.csv"
result.to_csv(save_path, index=False)
print(f"单帧数据处理完成，结果已保存为 {save_path}")


In [20]:
import sdf_helper as sh
import numpy as np
import pandas as pd
import os

# 设定数据文件的路径和文件名模式


base_path = '/media/lan/4248e56e-6d9a-4026-afad-e8b1d59ceab0/epoch/epoch3d/Cpf-Work/cone/cone1-a=145-0.75微米'
# base_path = '/media/lan/4248e56e-6d9a-4026-afad-e8b1d59ceab0/epoch/epoch3d/Cpf-Work/cone/cone1-a=145-4.5微米'
file_prefix = 'idall'
file_prefix_w = 'weight'
file_suffix = '.sdf'

# 存储结果的列表
results = []

# 创建一个 DataFrame 来保存中间结果
df_columns = [
    'File_Index', 
    'Lx_Electron', 'Ly_Electron', 'Lz_Electron', 
    'Lx_Photon', 'Ly_Photon', 'Lz_Photon', 
    'Total_Particle_Energy', 'Total_Laser_Energy_Injected',
    'Electron_Kinetic_Energy', 'Photon_Kinetic_Energy',
    'Electron_Count', 'Photon_Count'
]
df = pd.DataFrame(columns=df_columns)

for file_index in range(68):
    file_name = f"{file_prefix}{file_index:04d}{file_suffix}"  # 电子数据文件
    file_name_w = f"{file_prefix_w}{file_index:04d}{file_suffix}"  # 权重数据文件
    
    file_path = os.path.join(base_path, file_name)
    file_path_w = os.path.join(base_path, file_name_w)
    
    if not os.path.exists(file_path) or not os.path.exists(file_path_w):
        print(f"文件 {file_name} 或 {file_name_w} 不存在")
        continue
    
    try:
        Data_idall = sh.getdata(file_path)
        Data_w = sh.getdata(file_path_w)
        
        # 读取电子数据
        electron_x = Data_idall.Grid_Particles_subset_teste_Electron.data[0] 
        electron_y = Data_idall.Grid_Particles_subset_teste_Electron.data[1] 
        electron_z = Data_idall.Grid_Particles_subset_teste_Electron.data[2] 
        Electron_Px = Data_idall.Particles_Px_subset_teste_Electron.data
        Electron_Py = Data_idall.Particles_Py_subset_teste_Electron.data
        Electron_Pz = Data_idall.Particles_Pz_subset_teste_Electron.data
        Electron_weight = Data_w.Particles_Weight_subset_teste_Electron.data
        Ek_Electron = Data_idall.Particles_Ek_subset_teste_Electron.data

        # # 计算 x0
        # w0 = 7e-6  # w0 = 7 micron
        # x0 = 40e-6 + 2e-6 * np.exp(-(electron_z**2 + electron_y**2) / w0**2)        
        # # 过滤 x >= 20 的粒子
        # mask = (electron_x >= 20e-6) & (electron_x <= x0)  # x0 需要转换为微米
        mask = (electron_x >= 20e-6) & (electron_x <= 50e-6)  # x0 需要转换为微米

        filtered_X = electron_x[mask]
        filtered_Y = electron_y[mask]
        filtered_Z = electron_z[mask]
        filtered_Px = Electron_Px[mask]
        filtered_Py = Electron_Py[mask]
        filtered_Pz = Electron_Pz[mask]
        filtered_weight = Electron_weight[mask]
        filtered_Ek_Electron = Ek_Electron[mask]

        # 计算电子角动量
        Lx_Electron = np.sum((filtered_Y * filtered_Pz - filtered_Z * filtered_Py) * filtered_weight)
        Ly_Electron = np.sum((filtered_Z * filtered_Px - filtered_X * filtered_Pz) * filtered_weight)
        Lz_Electron = np.sum((filtered_X * filtered_Py - filtered_Y * filtered_Px) * filtered_weight)
        
        # 计算电子动能
        Electron_Kinetic_Energy = np.sum(filtered_Ek_Electron * filtered_weight)
        
        # 计算电子数量
        Electron_Count = np.sum(filtered_weight)

        # 尝试读取光子数据
        Lx_Photon, Ly_Photon, Lz_Photon = 0, 0, 0  # 默认值
        Photon_Kinetic_Energy = 0
        Photon_Count = 0
        try:
            # 读取光子数据
            photon_x = Data_idall.Grid_Particles_subset_testp_Photon.data[0] 
            photon_y = Data_idall.Grid_Particles_subset_testp_Photon.data[1] 
            photon_z = Data_idall.Grid_Particles_subset_testp_Photon.data[2] 
            Photon_Px = Data_idall.Particles_Px_subset_testp_Photon.data
            Photon_Py = Data_idall.Particles_Py_subset_testp_Photon.data
            Photon_Pz = Data_idall.Particles_Pz_subset_testp_Photon.data
            Photon_weight = Data_w.Particles_Weight_subset_testp_Photon.data
            Ek_Photon = Data_idall.Particles_Ek_subset_testp_Photon.data  # 假设存在光子动能数据

            # # 计算光子的 x0
            # photon_x0 = 40 + 2 * np.exp(-(photon_z**2 + photon_y**2) / w0**2)
            # # 过滤光子数据
            # photon_mask = (photon_x >= 20*1e-6) & (photon_x <= photon_x0*1e-6)  # x0 需要转换为微米
            photon_mask = (photon_x >= 20*1e-6) & (photon_x <= 50*1e-6)  # x0 需要转换为微米

            # 应用过滤
            filtered_photon_x = photon_x[photon_mask]
            filtered_photon_y = photon_y[photon_mask]
            filtered_photon_z = photon_z[photon_mask]
            filtered_Photon_Px = Photon_Px[photon_mask]
            filtered_Photon_Py = Photon_Py[photon_mask]
            filtered_Photon_Pz = Photon_Pz[photon_mask]
            filtered_Photon_weight = Photon_weight[photon_mask]
            filtered_Ek_Photon = Ek_Photon[photon_mask]

            # 计算光子角动量
            Lx_Photon = np.sum((filtered_photon_y * filtered_Photon_Pz - filtered_photon_z * filtered_Photon_Py) * filtered_Photon_weight)
            Ly_Photon = np.sum((filtered_photon_z * filtered_Photon_Px - filtered_photon_x * filtered_Photon_Pz) * filtered_Photon_weight)
            Lz_Photon = np.sum((filtered_photon_x * filtered_Photon_Py - filtered_photon_y * filtered_Photon_Px) * filtered_Photon_weight)
            
            # 计算光子动能
            Photon_Kinetic_Energy = np.sum(filtered_Ek_Photon * filtered_Photon_weight)
            
            # 计算光子数量
            Photon_Count = np.sum(filtered_Photon_weight)

        except AttributeError:
            # 如果光子数据不存在，捕获异常并继续
            print(f"光子数据在文件 {file_name} 中不存在")
        
        # 读取能量数据
        Total_Particle_Energy = Data_idall.Total_Particle_Energy_in_Simulation__J_.data / (1.602e-13)  # MeV
        Total_Laser_Energy_Injected = Data_idall.Absorption_Total_Laser_Energy_Injected__J_.data / (1.602e-13)  # MeV

        # 记录数据到列表
        results.append({
            'File_Index': file_index,
            'Lx_Electron': Lx_Electron,
            'Ly_Electron': Ly_Electron,
            'Lz_Electron': Lz_Electron,
            'Lx_Photon': Lx_Photon,
            'Ly_Photon': Ly_Photon,
            'Lz_Photon': Lz_Photon,
            'Total_Particle_Energy': Total_Particle_Energy,
            'Total_Laser_Energy_Injected': Total_Laser_Energy_Injected,
            'Electron_Kinetic_Energy': Electron_Kinetic_Energy,
            'Photon_Kinetic_Energy': Photon_Kinetic_Energy,
            'Electron_Count': Electron_Count,
            'Photon_Count': Photon_Count
        })
        
        print(f"文件 {file_name} 处理完成。")
    
    except Exception as e:
        print(f"处理文件 {file_name} 时发生错误: {e}")
    
# 将所有结果保存到 CSV 文件
df = pd.DataFrame(results)
df.to_csv("145-0.75微米角动量所有范围.csv", index=False)
print("所有计算完成，结果已保存到 cone1-a=145-0.5微米-angular_momentum_results.csv")

Reading file /media/lan/4248e56e-6d9a-4026-afad-e8b1d59ceab0/epoch/epoch3d/Cpf-Work/cone/cone1-a=145-0.75微米/idall0000.sdf
t() = time
px_subset_teste_electron(8743620,) = Particles/Px/subset_teste/Electron
py_subset_teste_electron(8743620,) = Particles/Py/subset_teste/Electron
pz_subset_teste_electron(8743620,) = Particles/Pz/subset_teste/Electron
ek_subset_teste_electron(8743620,) = Particles/Ek/subset_teste/Electron
id_subset_teste_electron(8743620,) = Particles/ID/subset_teste/Electron
x_grid_subset_teste_electron(8743620, 8743620, 8743620) = Grid_Particles_subset_teste_Electron x
y_grid_subset_teste_electron(8743620, 8743620, 8743620) = Grid_Particles_subset_teste_Electron y
z_grid_subset_teste_electron(8743620, 8743620, 8743620) = Grid_Particles_subset_teste_Electron z
Reading file /media/lan/4248e56e-6d9a-4026-afad-e8b1d59ceab0/epoch/epoch3d/Cpf-Work/cone/cone1-a=145-0.75微米/weight0000.sdf
t() = time
weight_subset_teste_electron(8743620,) = Particles/Weight/subset_teste/Electron
x_